In [1]:
import numpy as np
import glob
import random

from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.regularizers import l2
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

Using TensorFlow backend.
/Users/areed145/opt/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/areed145/opt/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/areed145/opt/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

In [2]:
input_files = glob.glob('arrays/*.npz')
print(len(input_files))

2600


In [3]:
Xs = []
ys = []

for file in input_files:
    loaded = np.load(file)
    Xs.append(loaded['X'])
    ys.append(loaded['y'])
    
X_ = np.array(Xs)
y_ = np.array(ys)

In [4]:
samples = 500
Xs_r = []
ys_r = []
for class_name in np.unique(y_):
    Xc_ = X_[y_==class_name]
    yc_ = y_[y_==class_name]
    for sample in range(samples):
        idx = random.randint(0,len(Xc_)-1)
        Xs_r.append(Xc_[idx])
        ys_r.append(yc_[idx])

In [5]:
X = np.array(Xs_r)
y = np.array(ys_r)
y = to_categorical(y)

In [6]:
print(len(X))

3500


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
shape = X.shape
print(X.shape)
print(X_train.shape)
print(X_test.shape)

categories = y.shape[1]
print(categories)

In [ ]:
model = Sequential([
    Conv2D(
        64, (3, 3),
        activation='relu',
        padding='same',
        input_shape=(shape[1], shape[2], shape[3]),
    ),
    Conv2D(
        64, (3, 3), 
        strides=(2,2),
        activation='relu',
        padding='same',
    ),
    MaxPooling2D(
        pool_size=2
    ),
    Dropout(0.25),
    Conv2D(
        128, (3, 3),
        activation='relu',
        padding='same',
    ),
    Conv2D(
        128, (3, 3), 
        strides=(2,2),
        activation='relu',
        padding='same',
    ),
    MaxPooling2D(
        pool_size=2
    ),
    Dropout(0.25),
    Flatten(),
    Dense(
        512, 
        activation='relu'
    ),
    Dropout(0.5),
    Dense(
        categories, 
        activation='softmax'
    ),
])

In [ ]:
model.compile(
    optimizer=SGD(lr=0.01),
#     optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train, y_train,
#     batch_size=100,
    epochs=50,
    validation_data=(X_test, y_test)
)

In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')
print('saved model to disk')